In [1]:
import json
from tqdm.auto import tqdm
from collections import defaultdict

In [2]:
item_to_user = defaultdict(list)
user_to_item = defaultdict(list)
with open('data/test.jsonl', 'r') as f:
    for row in tqdm(f, total=1671803):
        j = json.loads(row)
        session = j['session']
        for event in j['events']:
            item = event['aid']
            item_to_user[item].append(session)
            user_to_item[session].append(item)
            
item_to_user = dict(item_to_user)
user_to_item = dict(user_to_item)

  0%|          | 0/1671803 [00:00<?, ?it/s]

IOStream.flush timed out


In [3]:
for item, users in tqdm(item_to_user.items()):
    item_to_user[item] = set(users)
    
for user, items in tqdm(user_to_item.items()):
    user_to_item[user] = set(items)

  0%|          | 0/783486 [00:00<?, ?it/s]

  0%|          | 0/1671803 [00:00<?, ?it/s]

In [4]:
def arg_min(l):
    cur_min = l[0]
    arg = 0
    
    for i, elem in enumerate(l):
        if elem < cur_min:
            arg = i
            cur_min = elem
            
    return arg, cur_min

In [5]:
class ItemsCollections:
    def __init__(self, item_to_user: dict[int, list[int]], user_to_item,  k = 10):
        self._item_to_user = item_to_user
        self._user_to_item = user_to_item
        self._items = list(item_to_user.keys())
        
        self._top_k = dict()
        
        self._populest = list()
        self._fill_populest(k)
            
        # self._fill_top_k(k)
    
    @property
    def items(self):
        return self._items
    
    def get_sim(self, item1, item2):
        intersection = self._item_to_user[item1].intersection(self._item_to_user[item2])
        if len(intersection) == 0:
            return 0
        
        union = self._item_to_user[item1].union(self._item_to_user[item2])
        return len(intersection)/len(union)
    
    def _fill_populest(self, k):
        self._populest = [None] * k
        m = [0] * k
        
        cur_min = 0
        cur_arg = 0
        
        for item in tqdm(self._items):
            cur_pop = len(self._item_to_user[item])
            if cur_pop < cur_min:
                continue
                
            self._populest[cur_arg] = item
            m[cur_arg] = cur_pop
            cur_arg, cur_min = arg_min(m)
    
    def _fill_top_k(self, k):
        for item1 in tqdm(self._items):
            top_sim = [-1] * k
            top_item = list(self._populest)
            for i, item in enumerate(top_item):
                top_sim[i] = self.get_sim(item1, item)
            
            cur_min = 0
            cur_min_i = 0
            
            processed_items = set([item1])
            
            for user in self._item_to_user[item1]:
                for item2 in self._user_to_item[user]:
                    if item2 in processed_items:
                        continue
                        
                    processed_items.add(item2)

                    sim = self.get_sim(item1, item2)
                    if sim == 0:
                        continue

                    if sim > cur_min:
                        top_sim[cur_min_i] = sim
                        top_item[cur_min_i] = item2

                        cur_min_i, cur_min = arg_min(top_sim)
                    
            self._top_k[item1] = set(top_item)
    
    
items = ItemsCollections(item_to_user, user_to_item)
#ItemsCollections_top_k = items._top_k
items._top_k = ItemsCollections_top_k

  0%|          | 0/783486 [00:00<?, ?it/s]

  0%|          | 0/783486 [00:00<?, ?it/s]

In [71]:
for user in user_to_item.keys():
    if len(user_to_item[user]) == 10:
        break

In [81]:
%%time
C = set()
for hist_item in user_to_item[user]:
    C = C.union(items._top_k[hist_item])
    
C = list(C)
sims = [0] * len(C)
for i, cand in enumerate(C):
    if cand in user_to_item[user]:
        continue
    
    for hist_item in user_to_item[user]:
        sims[i] += items.get_sim(hist_item, cand)

CPU times: user 9.94 ms, sys: 0 ns, total: 9.94 ms
Wall time: 9.06 ms


In [88]:
import kaggle

In [89]:
kaggle.api.authenticate()

In [91]:
kaggle.api.dataset_download_files('otto/recsys-dataset', path='data', unzip=True, quiet=False)

  0%|          | 4.00M/1.98G [00:06<54:35, 648kB/s]  


KeyboardInterrupt: 